# KAN (Kolmogorov-Arnold Networks) 求解 PDE 教程

## 📚 教程概述

本教程将带你深入了解如何使用 **Kolmogorov-Arnold Networks (KAN)** 求解偏微分方程（PDE）。

### 学习内容

1. **理论基础**: Kolmogorov-Arnold 表示定理
2. **核心技术**: B-spline 基函数的实现
3. **网络构建**: 从零开始构建 KAN 层和网络
4. **PDE 求解**: 以 1D Poisson 方程为例
5. **可视化分析**: 理解 KAN 学到的函数

### Kolmogorov-Arnold 表示定理

任何多元连续函数 $f: [0,1]^n \to \mathbb{R}$ 都可以表示为：

$$
f(x_1, \dots, x_n) = \sum_{q=0}^{2n} \Phi_q\left(\sum_{p=1}^{n} \phi_{q,p}(x_p)\right)
$$

其中 $\Phi_q$ 和 $\phi_{q,p}$ 都是**一元连续函数**。

**KAN 的核心思想**：用可学习的 B-spline 基函数替代传统神经网络的固定激活函数！

---

### 本教程使用的 PDE 问题

我们将求解 1D Poisson 方程：

$$
-\frac{d^2u}{dx^2} = \sin(\pi x), \quad x \in [-1, 1]
$$

**边界条件**: $u(-1) = u(1) = 0$

**解析解**: $u(x) = \frac{\sin(\pi x)}{\pi^2}$

## 1. 导入必要的库

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子以确保结果可复现
torch.manual_seed(42)
np.random.seed(42)

# 设置绘图风格
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文显示
plt.rcParams['axes.unicode_minus'] = False

print("✓ 库导入成功！")
print(f"PyTorch 版本: {torch.__version__}")
print(f"设备: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

## 2. 实现 B-spline 基函数

B-spline（B样条）是 KAN 的核心组件。它是一系列分段多项式函数，具有良好的数值稳定性和局部性。

### 数学定义

B-spline 基函数 $B_i^k(x)$ 可以通过 Cox-de Boor 递推公式计算：

**0阶 (常数)**:
$$
B_i^0(x) = \begin{cases}
1, & \text{if } t_i \leq x < t_{i+1} \\
0, & \text{otherwise}
\end{cases}
$$

**k阶递推**:
$$
B_i^k(x) = \frac{x - t_i}{t_{i+k} - t_i}B_i^{k-1}(x) + \frac{t_{i+k+1} - x}{t_{i+k+1} - t_{i+1}}B_{i+1}^{k-1}(x)
$$

其中 $\{t_i\}$ 是节点向量（knot vector）。

In [ ]:
class BSpline:
    """
    B-spline 基函数实现
    使用 Cox-de Boor 递推公式
    """
    def __init__(self, grid_size=5, spline_order=3, x_min=-1.0, x_max=1.0):
        """
        Args:
            grid_size: 网格点数量
            spline_order: B-spline 阶数 (通常为3，对应3次样条)
            x_min, x_max: 定义域范围
        """
        self.grid_size = grid_size
        self.order = spline_order
        self.x_min = x_min
        self.x_max = x_max
        
        # 创建节点向量 (knot vector)
        # 使用均匀节点，两端重复节点以保证边界插值
        self.knots = self._create_knots()
        
    def _create_knots(self):
        """创建均匀节点向量"""
        # 内部均匀节点
        interior_knots = torch.linspace(self.x_min, self.x_max, self.grid_size)
        
        # 两端重复节点
        left_knots = torch.ones(self.order) * self.x_min
        right_knots = torch.ones(self.order) * self.x_max
        
        # 组合
        knots = torch.cat([left_knots, interior_knots, right_knots])
        return knots
    
    def basis(self, x, i, k):
        """
        计算第i个k阶B-spline基函数在x处的值
        使用Cox-de Boor递推公式
        
        Args:
            x: 输入点 (tensor)
            i: 基函数索引
            k: B-spline阶数
        
        Returns:
            基函数值
        """
        # 0阶基函数
        if k == 0:
            return ((x >= self.knots[i]) & (x < self.knots[i+1])).float()
        
        # 递推计算
        # 第一项
        denom1 = self.knots[i+k] - self.knots[i]
        term1 = 0.0
        if denom1 > 1e-10:  # 避免除零
            term1 = (x - self.knots[i]) / denom1 * self.basis(x, i, k-1)
        
        # 第二项
        denom2 = self.knots[i+k+1] - self.knots[i+1]
        term2 = 0.0
        if denom2 > 1e-10:
            term2 = (self.knots[i+k+1] - x) / denom2 * self.basis(x, i+1, k-1)
        
        return term1 + term2
    
    def __call__(self, x):
        """
        计算所有基函数在x处的值
        
        Args:
            x: 输入点 [batch_size] 或 [batch_size, 1]
        
        Returns:
            basis_values: [batch_size, num_basis] 所有基函数的值
        """
        if x.dim() == 1:
            x = x.unsqueeze(-1)
        
        # 夹紧到定义域内
        x = torch.clamp(x, self.x_min, self.x_max)
        
        # 计算所有基函数
        num_basis = len(self.knots) - self.order - 1
        basis_values = []
        
        for i in range(num_basis):
            basis_i = self.basis(x.squeeze(-1), i, self.order)
            basis_values.append(basis_i)
        
        return torch.stack(basis_values, dim=-1)  # [batch_size, num_basis]

# 测试 B-spline 实现
print("=" * 60)
print("B-spline 基函数测试")
print("=" * 60)

bspline = BSpline(grid_size=5, spline_order=3, x_min=-1, x_max=1)
print(f"网格点数: {bspline.grid_size}")
print(f"B-spline阶数: {bspline.order}")
print(f"节点向量: {bspline.knots}")
print(f"基函数数量: {len(bspline.knots) - bspline.order - 1}")

# 可视化 B-spline 基函数
x_test = torch.linspace(-1, 1, 200)
basis_values = bspline(x_test)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
for i in range(basis_values.shape[1]):
    plt.plot(x_test.numpy(), basis_values[:, i].numpy(), 
             label=f'$B_{i}^{bspline.order}(x)$', linewidth=2)
plt.xlabel('x', fontsize=12)
plt.ylabel('Basis Value', fontsize=12)
plt.title('B-spline 基函数 (3次样条)', fontsize=14, fontweight='bold')
plt.legend(ncol=2, fontsize=9)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(x_test.numpy(), basis_values.sum(dim=1).numpy(), 
         'r-', linewidth=3, label='所有基函数之和')
plt.axhline(y=1.0, color='k', linestyle='--', alpha=0.5, label='y=1')
plt.xlabel('x', fontsize=12)
plt.ylabel('Sum of Basis', fontsize=12)
plt.title('单位分解性质', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ B-spline 基函数实现完成！")
print("  性质验证：所有基函数之和恒等于1（单位分解性）")

## 3. 构建 KAN 层

KAN 层将多个输入通过可学习的一元函数（B-spline）进行变换。

### KAN 层的数学表达

对于输入 $\mathbf{x} = [x_1, \dots, x_n]$，KAN 层的第 $j$ 个输出为：

$$
y_j = \sum_{i=1}^{n} \Phi_{i,j}(x_i) = \sum_{i=1}^{n} \sum_{k=1}^{G} c_{i,j,k} B_k(x_i)
$$

其中：
- $c_{i,j,k}$ 是可学习的 B-spline 系数
- $B_k$ 是第 $k$ 个 B-spline 基函数
- $G$ 是基函数的数量

In [ ]:
class KANLayer(nn.Module):
    """
    Kolmogorov-Arnold Network 层
    使用 B-spline 基函数作为可学习的激活函数
    """
    def __init__(self, in_features, out_features, grid_size=5, spline_order=3):
        """
        Args:
            in_features: 输入维度
            out_features: 输出维度
            grid_size: B-spline 网格大小
            spline_order: B-spline 阶数
        """
        super(KANLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.grid_size = grid_size
        self.spline_order = spline_order
        
        # 创建 B-spline 基函数
        self.bspline = BSpline(grid_size, spline_order)
        self.num_basis = len(self.bspline.knots) - spline_order - 1
        
        # 可学习的 B-spline 系数
        # 形状: [in_features, out_features, num_basis]
        self.coefficients = nn.Parameter(
            torch.randn(in_features, out_features, self.num_basis) * 0.1
        )
        
        # 可选的残差连接权重
        self.residual_weight = nn.Parameter(torch.randn(in_features, out_features) * 0.01)
        
    def forward(self, x):
        """
        前向传播
        
        Args:
            x: [batch_size, in_features]
        
        Returns:
            y: [batch_size, out_features]
        """
        batch_size = x.shape[0]
        
        # 对每个输入维度计算 B-spline 基函数值
        # x[:, i] -> [batch_size] -> basis -> [batch_size, num_basis]
        basis_values = []
        for i in range(self.in_features):
            basis_i = self.bspline(x[:, i])  # [batch_size, num_basis]
            basis_values.append(basis_i)
        
        # basis_all: [batch_size, in_features, num_basis]
        basis_all = torch.stack(basis_values, dim=1)
        
        # 计算输出: y_j = sum_i sum_k c_{i,j,k} * B_k(x_i)
        # 使用 einsum 进行高效计算
        # basis_all: [batch, in, num_basis]
        # coefficients: [in, out, num_basis]
        # output: [batch, out]
        output = torch.einsum('bin,ion->bo', basis_all, self.coefficients)
        
        # 添加残差连接 (可选)
        residual = torch.matmul(x, self.residual_weight)
        output = output + residual
        
        return output
    
    def extra_repr(self):
        return f'in_features={self.in_features}, out_features={self.out_features}, ' \
               f'grid_size={self.grid_size}, spline_order={self.spline_order}'


# 测试 KAN 层
print("=" * 60)
print("KAN 层测试")
print("=" * 60)

# 创建一个简单的 KAN 层: 1输入 -> 3输出
kan_layer = KANLayer(in_features=1, out_features=3, grid_size=5, spline_order=3)
print(kan_layer)
print(f"\n参数数量: {sum(p.numel() for p in kan_layer.parameters())}")

# 测试前向传播
x_test = torch.linspace(-1, 1, 100).unsqueeze(1)  # [100, 1]
y_test = kan_layer(x_test)  # [100, 3]

print(f"输入形状: {x_test.shape}")
print(f"输出形状: {y_test.shape}")

# 可视化学习到的函数
plt.figure(figsize=(10, 4))
for i in range(3):
    plt.plot(x_test.detach().numpy(), y_test[:, i].detach().numpy(), 
             label=f'Output {i+1}', linewidth=2)

plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('KAN 层输出（随机初始化）', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✓ KAN 层实现完成！")

## 4. 构建完整的 KAN 网络

现在我们将多个 KAN 层堆叠起来，构建一个深度 KAN 网络用于求解 PDE。

In [ ]:
class KANPDE(nn.Module):
    """
    用于求解 PDE 的 KAN 网络
    """
    def __init__(self, layers, grid_size=5, spline_order=3):
        """
        Args:
            layers: 层维度列表，例如 [1, 20, 20, 1] 表示输入1维，两个隐藏层各20维，输出1维
            grid_size: B-spline 网格大小
            spline_order: B-spline 阶数
        """
        super(KANPDE, self).__init__()
        self.layers_dims = layers
        self.depth = len(layers) - 1
        
        # 构建 KAN 层
        self.kan_layers = nn.ModuleList()
        for i in range(self.depth):
            layer = KANLayer(
                in_features=layers[i],
                out_features=layers[i+1],
                grid_size=grid_size,
                spline_order=spline_order
            )
            self.kan_layers.append(layer)
    
    def forward(self, x):
        """
        前向传播
        
        Args:
            x: [batch_size, input_dim] 或 [batch_size]
        
        Returns:
            y: [batch_size, output_dim]
        """
        # 确保输入是二维的
        if x.dim() == 1:
            x = x.unsqueeze(1)
        
        # 逐层传播
        for layer in self.kan_layers:
            x = layer(x)
        
        return x
    
    def count_parameters(self):
        """统计参数数量"""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)
    
    def regularization_loss(self, lambda_reg=1e-4):
        """
        计算正则化损失（L2正则）
        鼓励 B-spline 系数保持较小，避免过拟合
        """
        reg_loss = 0.0
        for layer in self.kan_layers:
            reg_loss += torch.sum(layer.coefficients ** 2)
        return lambda_reg * reg_loss


# 创建 KAN 网络
print("=" * 60)
print("构建 KAN 网络")
print("=" * 60)

# 网络架构: 1D 输入 -> 20维 -> 20维 -> 20维 -> 1D 输出
model = KANPDE(layers=[1, 20, 20, 20, 1], grid_size=5, spline_order=3)

print(f"网络架构: {model.layers_dims}")
print(f"网络深度: {model.depth} 层")
print(f"总参数量: {model.count_parameters()}")
print("\n网络结构:")
print(model)

# 测试网络
x_test = torch.linspace(-1, 1, 100).unsqueeze(1)
with torch.no_grad():
    y_test = model(x_test)

plt.figure(figsize=(10, 4))
plt.plot(x_test.numpy(), y_test.numpy(), 'b-', linewidth=2, label='KAN输出（未训练）')
plt.xlabel('x', fontsize=12)
plt.ylabel('u(x)', fontsize=12)
plt.title('KAN 网络输出（随机初始化）', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✓ KAN 网络构建完成！")

## 5. 生成训练数据

对于 PDE 问题:
$$-\frac{d^2u}{dx^2} = \sin(\pi x), \quad x \in [-1, 1]$$
$$u(-1) = u(1) = 0$$

我们需要：
1. **配点（Collocation Points）**: 内部点，用于满足 PDE
2. **边界点（Boundary Points）**: 边界点，用于满足边界条件

In [ ]:
# 解析解
def analytical_solution(x):
    """1D Poisson 方程的解析解"""
    return torch.sin(np.pi * x) / (np.pi ** 2)

# 生成训练数据
n_collocation = 100  # 内部配点数
n_boundary = 2       # 边界点数（左右各1个）

# 配点（内部点）
x_collocation = torch.linspace(-1, 1, n_collocation).unsqueeze(1)
x_collocation.requires_grad = True

# 边界点
x_boundary = torch.tensor([[-1.0], [1.0]])
u_boundary = torch.tensor([[0.0], [0.0]])  # 边界条件 u(-1) = u(1) = 0

# 可视化训练数据
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.scatter(x_collocation.detach().numpy(), 
           torch.zeros_like(x_collocation).numpy(), 
           c='blue', s=30, alpha=0.6, label=f'配点 (n={n_collocation})')
plt.scatter(x_boundary.numpy(), u_boundary.numpy(), 
           c='red', s=100, marker='s', label=f'边界点 (n={n_boundary})', zorder=5)
plt.xlabel('x', fontsize=12)
plt.ylabel('u', fontsize=12)
plt.title('训练数据分布', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
x_plot = torch.linspace(-1, 1, 200)
u_exact = analytical_solution(x_plot)
plt.plot(x_plot.numpy(), u_exact.numpy(), 'g-', linewidth=3, label='解析解')
plt.scatter(x_boundary.numpy(), u_boundary.numpy(), 
           c='red', s=100, marker='s', label='边界条件', zorder=5)
plt.xlabel('x', fontsize=12)
plt.ylabel('u(x)', fontsize=12)
plt.title('解析解', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("=" * 60)
print("训练数据统计")
print("=" * 60)
print(f"配点数量: {n_collocation}")
print(f"边界点数量: {n_boundary}")
print(f"总数据点: {n_collocation + n_boundary}")
print(f"定义域: [-1, 1]")
print("=" * 60)

## 6. 定义 PDE 损失函数

损失函数包含三部分：
1. **PDE 残差损失**: $\mathcal{L}_{\text{PDE}} = \frac{1}{N}\sum \left|-u_{xx} - \sin(\pi x)\right|^2$
2. **边界条件损失**: $\mathcal{L}_{\text{BC}} = \frac{1}{2}\sum \left|u(x_b) - 0\right|^2$
3. **正则化损失**: $\mathcal{L}_{\text{reg}} = \lambda \sum c_{ijk}^2$

**总损失**: $\mathcal{L} = \mathcal{L}_{\text{PDE}} + \lambda_{\text{BC}} \mathcal{L}_{\text{BC}} + \mathcal{L}_{\text{reg}}$

In [ ]:
def compute_loss(model, x_col, x_bc, u_bc, lambda_bc=10.0):
    """
    计算总损失
    
    Args:
        model: KAN 模型
        x_col: 配点 [n_col, 1]
        x_bc: 边界点 [n_bc, 1]
        u_bc: 边界值 [n_bc, 1]
        lambda_bc: 边界条件损失权重
    
    Returns:
        total_loss: 总损失
        loss_dict: 各项损失字典
    """
    # 1. PDE 残差损失
    u_pred = model(x_col)
    
    # 计算一阶导数 du/dx
    u_x = torch.autograd.grad(
        outputs=u_pred,
        inputs=x_col,
        grad_outputs=torch.ones_like(u_pred),
        create_graph=True,
        retain_graph=True
    )[0]
    
    # 计算二阶导数 d²u/dx²
    u_xx = torch.autograd.grad(
        outputs=u_x,
        inputs=x_col,
        grad_outputs=torch.ones_like(u_x),
        create_graph=True,
        retain_graph=True
    )[0]
    
    # PDE: -u_xx = sin(πx)
    f = torch.sin(np.pi * x_col)
    pde_residual = -u_xx - f
    loss_pde = torch.mean(pde_residual ** 2)
    
    # 2. 边界条件损失
    u_bc_pred = model(x_bc)
    loss_bc = torch.mean((u_bc_pred - u_bc) ** 2)
    
    # 3. 正则化损失
    loss_reg = model.regularization_loss()
    
    # 总损失
    total_loss = loss_pde + lambda_bc * loss_bc + loss_reg
    
    loss_dict = {
        'total': total_loss.item(),
        'pde': loss_pde.item(),
        'bc': loss_bc.item(),
        'reg': loss_reg.item()
    }
    
    return total_loss, loss_dict

# 测试损失函数
print("=" * 60)
print("损失函数测试")
print("=" * 60)

with torch.no_grad():
    # 临时启用梯度
    x_collocation.requires_grad = True
    
loss, loss_dict = compute_loss(model, x_collocation, x_boundary, u_boundary)

print(f"总损失: {loss_dict['total']:.6f}")
print(f"  - PDE 残差损失: {loss_dict['pde']:.6f}")
print(f"  - 边界条件损失: {loss_dict['bc']:.6f}")
print(f"  - 正则化损失: {loss_dict['reg']:.6f}")
print("=" * 60)
print("✓ 损失函数定义完成！")

## 7. 训练 KAN 模型

使用 Adam 优化器训练模型，监控损失下降过程。

In [ ]:
# 训练参数
epochs = 5000
learning_rate = 1e-3
print_every = 500

# 优化器
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 学习率调度器
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=500, verbose=False
)

# 记录训练历史
history = {
    'total': [],
    'pde': [],
    'bc': [],
    'reg': []
}

print("=" * 60)
print("开始训练 KAN 模型".center(60))
print("=" * 60)
print(f"训练轮数: {epochs}")
print(f"学习率: {learning_rate}")
print(f"模型参数: {model.count_parameters()}")
print("=" * 60)

# 训练循环
for epoch in range(epochs):
    optimizer.zero_grad()
    
    # 计算损失
    loss, loss_dict = compute_loss(model, x_collocation, x_boundary, u_boundary)
    
    # 反向传播
    loss.backward()
    
    # 梯度裁剪（防止梯度爆炸）
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    # 更新参数
    optimizer.step()
    
    # 更新学习率
    scheduler.step(loss)
    
    # 记录历史
    for key in history:
        history[key].append(loss_dict[key])
    
    # 打印进度
    if (epoch + 1) % print_every == 0 or epoch == 0:
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch+1:4d} | LR: {current_lr:.2e} | "
              f"Total: {loss_dict['total']:.6f} | "
              f"PDE: {loss_dict['pde']:.6f} | "
              f"BC: {loss_dict['bc']:.6f}")

print("=" * 60)
print("训练完成！".center(60))
print("=" * 60)

# 绘制训练历史
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 总损失
axes[0].semilogy(history['total'], 'b-', linewidth=2, label='总损失')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss (log scale)', fontsize=12)
axes[0].set_title('训练损失曲线', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].legend(fontsize=11)

# 各项损失
axes[1].semilogy(history['pde'], label='PDE 残差', linewidth=2, alpha=0.8)
axes[1].semilogy(history['bc'], label='边界条件', linewidth=2, alpha=0.8)
axes[1].semilogy(history['reg'], label='正则化', linewidth=2, alpha=0.8)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss (log scale)', fontsize=12)
axes[1].set_title('各项损失分解', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend(fontsize=11)

plt.tight_layout()
plt.show()

print(f"\n最终损失: {history['total'][-1]:.6e}")

## 8. 可视化结果与误差分析

比较 KAN 预测解与解析解，计算误差指标。

In [ ]:
# 评估模型
x_eval = torch.linspace(-1, 1, 300).unsqueeze(1)

with torch.no_grad():
    u_pred = model(x_eval).squeeze()
    u_exact = analytical_solution(x_eval.squeeze())

# 计算误差
error = torch.abs(u_pred - u_exact)
l2_error = torch.sqrt(torch.mean(error ** 2))
linf_error = torch.max(error)
relative_l2 = l2_error / torch.sqrt(torch.mean(u_exact ** 2))

print("=" * 60)
print("误差分析".center(60))
print("=" * 60)
print(f"L2 误差:           {l2_error:.6e}")
print(f"L∞ 误差:           {linf_error:.6e}")
print(f"相对 L2 误差:      {relative_l2:.6e} ({relative_l2*100:.4f}%)")
print("=" * 60)

# 可视化
fig = plt.figure(figsize=(16, 10))

# 1. 解的比较
ax1 = fig.add_subplot(2, 3, 1)
ax1.plot(x_eval.numpy(), u_exact.numpy(), 'g-', linewidth=3, 
         label='解析解', alpha=0.7)
ax1.plot(x_eval.numpy(), u_pred.numpy(), 'b--', linewidth=2, 
         label='KAN 预测')
ax1.scatter(x_boundary.numpy(), u_boundary.numpy(), 
           c='red', s=100, marker='s', label='边界条件', zorder=5)
ax1.set_xlabel('x', fontsize=12)
ax1.set_ylabel('u(x)', fontsize=12)
ax1.set_title('解的比较', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# 2. 误差分布
ax2 = fig.add_subplot(2, 3, 2)
ax2.plot(x_eval.numpy(), error.numpy(), 'r-', linewidth=2)
ax2.fill_between(x_eval.numpy().squeeze(), 0, error.numpy(), alpha=0.3, color='red')
ax2.set_xlabel('x', fontsize=12)
ax2.set_ylabel('|误差|', fontsize=12)
ax2.set_title(f'绝对误差 (L2={l2_error:.2e})', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# 3. 相对误差
ax3 = fig.add_subplot(2, 3, 3)
relative_error = (error / torch.abs(u_exact + 1e-10)) * 100
ax3.plot(x_eval.numpy(), relative_error.numpy(), 'purple', linewidth=2)
ax3.set_xlabel('x', fontsize=12)
ax3.set_ylabel('相对误差 (%)', fontsize=12)
ax3.set_title('相对误差分布', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)

# 4. PDE 残差验证
ax4 = fig.add_subplot(2, 3, 4)
x_test = torch.linspace(-1, 1, 100).unsqueeze(1)
x_test.requires_grad = True

u_test = model(x_test)
u_x_test = torch.autograd.grad(u_test, x_test, 
                               grad_outputs=torch.ones_like(u_test),
                               create_graph=True)[0]
u_xx_test = torch.autograd.grad(u_x_test, x_test,
                                grad_outputs=torch.ones_like(u_x_test),
                                create_graph=True)[0]

pde_residual = (-u_xx_test - torch.sin(np.pi * x_test)).detach()

ax4.plot(x_test.detach().numpy(), pde_residual.numpy(), 'orange', linewidth=2)
ax4.axhline(y=0, color='k', linestyle='--', alpha=0.5)
ax4.set_xlabel('x', fontsize=12)
ax4.set_ylabel('PDE 残差', fontsize=12)
ax4.set_title('PDE 残差: $-u_{xx} - \\sin(\\pi x)$', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)

# 5. 二阶导数比较
ax5 = fig.add_subplot(2, 3, 5)
u_xx_exact = -(np.pi ** 2) * torch.sin(np.pi * x_test.detach())
ax5.plot(x_test.detach().numpy(), u_xx_exact.numpy(), 'g-', 
         linewidth=3, label='解析解', alpha=0.7)
ax5.plot(x_test.detach().numpy(), u_xx_test.detach().numpy(), 'b--', 
         linewidth=2, label='KAN')
ax5.set_xlabel('x', fontsize=12)
ax5.set_ylabel('$u_{xx}$', fontsize=12)
ax5.set_title('二阶导数比较', fontsize=14, fontweight='bold')
ax5.legend(fontsize=11)
ax5.grid(True, alpha=0.3)

# 6. 误差统计直方图
ax6 = fig.add_subplot(2, 3, 6)
ax6.hist(error.numpy(), bins=30, color='skyblue', edgecolor='black', alpha=0.7)
ax6.axvline(x=l2_error.item(), color='r', linestyle='--', linewidth=2, 
           label=f'L2={l2_error:.2e}')
ax6.set_xlabel('误差大小', fontsize=12)
ax6.set_ylabel('频数', fontsize=12)
ax6.set_title('误差分布直方图', fontsize=14, fontweight='bold')
ax6.legend(fontsize=11)
ax6.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n✓ 结果可视化完成！")

## 9. 网格细化与自适应训练

KAN 的一大优势是可以通过增加 B-spline 网格点来提高精度。

In [ ]:
# 创建多个不同网格大小的模型进行比较
grid_sizes = [3, 5, 7, 9]
models_comparison = []
errors_comparison = []

print("=" * 60)
print("网格细化实验".center(60))
print("=" * 60)

for grid_size in grid_sizes:
    print(f"\n训练网格大小 = {grid_size} 的模型...")
    
    # 创建模型
    model_grid = KANPDE(layers=[1, 20, 20, 20, 1], 
                       grid_size=grid_size, 
                       spline_order=3)
    
    # 优化器
    optimizer_grid = torch.optim.Adam(model_grid.parameters(), lr=1e-3)
    
    # 快速训练
    for epoch in range(3000):
        optimizer_grid.zero_grad()
        loss, _ = compute_loss(model_grid, x_collocation, x_boundary, u_boundary)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_grid.parameters(), max_norm=1.0)
        optimizer_grid.step()
    
    # 评估
    with torch.no_grad():
        u_pred_grid = model_grid(x_eval).squeeze()
        error_grid = torch.abs(u_pred_grid - u_exact)
        l2_error_grid = torch.sqrt(torch.mean(error_grid ** 2))
    
    models_comparison.append(model_grid)
    errors_comparison.append(l2_error_grid.item())
    
    print(f"  参数量: {model_grid.count_parameters()}")
    print(f"  L2 误差: {l2_error_grid:.6e}")

print("\n" + "=" * 60)
print("网格细化结果".center(60))
print("=" * 60)
for i, gs in enumerate(grid_sizes):
    print(f"网格大小 {gs}: L2误差 = {errors_comparison[i]:.6e}")
print("=" * 60)

# 可视化比较
fig = plt.figure(figsize=(16, 10))

# 1. 误差 vs 网格大小
ax1 = fig.add_subplot(2, 3, 1)
ax1.semilogy(grid_sizes, errors_comparison, 'o-', linewidth=2, markersize=10)
ax1.set_xlabel('网格大小', fontsize=12)
ax1.set_ylabel('L2 误差 (log scale)', fontsize=12)
ax1.set_title('网格细化效果', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# 2-5. 不同网格大小的解
for idx, (gs, model_grid) in enumerate(zip(grid_sizes, models_comparison)):
    ax = fig.add_subplot(2, 3, idx+2)
    with torch.no_grad():
        u_pred_grid = model_grid(x_eval).squeeze()
    
    ax.plot(x_eval.numpy(), u_exact.numpy(), 'g-', linewidth=2, 
           label='解析解', alpha=0.7)
    ax.plot(x_eval.numpy(), u_pred_grid.numpy(), 'b--', linewidth=2,
           label='KAN')
    ax.set_xlabel('x', fontsize=11)
    ax.set_ylabel('u(x)', fontsize=11)
    ax.set_title(f'网格大小 = {gs} (L2={errors_comparison[idx]:.2e})', 
                fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ 网格细化实验完成！")
print("  结论: 网格越细，精度越高（但计算成本也增加）")

## 10. 可视化学习到的一元函数

KAN 的可解释性体现在：我们可以直接查看每层学到的一元函数！

In [ ]:
# 提取并可视化每层的一元函数
def visualize_univariate_functions(model, layer_idx=0, num_functions=5):
    """
    可视化 KAN 层中学到的一元函数
    
    Args:
        model: KANPDE 模型
        layer_idx: 要可视化的层索引
        num_functions: 要显示的函数数量
    """
    layer = model.kan_layers[layer_idx]
    
    # 创建测试输入
    x_test = torch.linspace(-1, 1, 200)
    
    # 计算 B-spline 基函数值
    basis_values = layer.bspline(x_test)  # [200, num_basis]
    
    # 选择部分输入-输出对进行可视化
    in_dim = min(layer.in_features, 1)  # 第一层只有1个输入
    out_dim = min(layer.out_features, num_functions)
    
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    axes = axes.flatten()
    
    for out_idx in range(min(out_dim, 6)):
        ax = axes[out_idx]
        
        # 对于第一个输入维度，计算对应的一元函数
        # f(x) = sum_k c_{0, out_idx, k} * B_k(x)
        coeffs = layer.coefficients[0, out_idx, :].detach()  # [num_basis]
        univariate_func = torch.matmul(basis_values, coeffs)  # [200]
        
        ax.plot(x_test.numpy(), univariate_func.numpy(), 'b-', linewidth=2)
        ax.set_xlabel('x', fontsize=11)
        ax.set_ylabel('$\phi(x)$', fontsize=11)
        ax.set_title(f'第{layer_idx+1}层: 输入0 → 输出{out_idx}', 
                    fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
        ax.axvline(x=0, color='k', linestyle='--', alpha=0.3)
    
    # 删除多余的子图
    for idx in range(out_dim, 6):
        fig.delaxes(axes[idx])
    
    plt.suptitle(f'KAN 第{layer_idx+1}层学到的一元函数', 
                fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()


print("=" * 60)
print("可视化学习到的一元函数".center(60))
print("=" * 60)

# 可视化第一层的一元函数
print("\n第1层 (1 → 20):")
visualize_univariate_functions(model, layer_idx=0, num_functions=6)

# 可视化最后一层的一元函数
print(f"\n第{model.depth}层 (20 → 1):")
if model.depth > 1:
    # 对于最后一层，我们可视化几个输入到输出的映射
    layer_last = model.kan_layers[-1]
    x_test = torch.linspace(-1, 1, 200)
    basis_values = layer_last.bspline(x_test)
    
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    axes = axes.flatten()
    
    # 可视化前6个输入神经元到输出的贡献
    for in_idx in range(min(6, layer_last.in_features)):
        ax = axes[in_idx]
        coeffs = layer_last.coefficients[in_idx, 0, :].detach()
        univariate_func = torch.matmul(basis_values, coeffs)
        
        ax.plot(x_test.numpy(), univariate_func.numpy(), 'r-', linewidth=2)
        ax.set_xlabel('隐藏层激活值', fontsize=11)
        ax.set_ylabel('贡献', fontsize=11)
        ax.set_title(f'最后一层: 输入{in_idx} → 输出', 
                    fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    
    plt.suptitle(f'KAN 最后一层学到的一元函数 (组合成最终输出)', 
                fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

print("\n✓ 一元函数可视化完成！")
print("  KAN 通过组合这些一元函数来逼近复杂的 PDE 解！")

## 总结与展望

### 🎯 本教程学到的内容

1. **理论基础**: Kolmogorov-Arnold 表示定理将多元函数分解为一元函数的组合
2. **B-spline 实现**: 使用 Cox-de Boor 递推公式实现 B-spline 基函数
3. **KAN 层构建**: 用 B-spline 系数作为可学习参数，替代传统激活函数
4. **PDE 求解**: 通过 PDE 残差 + 边界条件损失训练 KAN
5. **网格细化**: 增加 B-spline 网格点提高精度
6. **可解释性**: 可视化学习到的一元函数，理解模型行为

### ✨ KAN 的优势

- **参数效率**: 相比 MLP 需要更少参数达到相同精度
- **可解释性**: 可以直接查看和分析学习到的一元函数
- **灵活性**: 支持自适应网格细化
- **精度**: 在光滑函数逼近上表现优异

### ⚠️ KAN 的局限性

- **计算成本**: B-spline 计算比简单激活函数慢
- **初始化敏感**: 系数初始化会影响收敛
- **不适合间断解**: 对于激波等间断问题表现较差
- **实现复杂**: 相比标准 MLP 实现更复杂

### 📚 进阶方向

1. **时空 PDE**: 求解时间依赖的 PDE（如热传导、Burgers 方程）
2. **高维问题**: 扩展到 2D/3D PDE
3. **自适应采样**: 在误差大的区域增加配点
4. **符号回归**: 提取可解释的数学表达式
5. **与其他方法结合**: KAN + FNO, KAN + DeepONet

### 🔗 相关资源

- 项目示例代码: `../examples/`
- 更多 PDE 求解方法: `../../PINNs/`, `../../DeepONet/`, `../../FNO/`
- 原始论文: Liu et al. (2024) "KAN: Kolmogorov-Arnold Networks"

---

**感谢使用本教程！如有问题欢迎交流。** 🚀